Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/automated-machine-learning/forecasting-orange-juice-sales/auto-ml-forecasting-orange-juice-sales.png)

# Synthetic Data Generation
In this notebook, we are generating a large data set to demonstrate the capabilities of the ParallelRunStep. The dataset is from the Univeristy of Chicago’s Dominick’s Finer Foods for orange juice sales. Dominick's is a grocery chain in the Chicago metropolitan area. The original data has ~130 stores. We generated an additional 4,000 stores. 

We synthetically generate a total of 12,222 orange juice sales datasets and leverage ParallelRunStep to build many models to predict the quantity/revenue per store per brand. Each dataset contains ~120 rows with over 2 years data. 

- [ParallelRunStep Documentation](https://docs.microsoft.com/en-us/python/api/azureml-contrib-pipeline-steps/azureml.contrib.pipeline.steps.parallelrunstep?view=azure-ml-py)

## Setup

From the 00_Setup_Env notebook, you've set up a AML Workspace where we will build our pipelines.

In [2]:
import azureml.core
import pandas as pd
import numpy as np
import logging
import random

from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.dataset import Dataset

In [2]:
ws = Workspace(subscription_id="bbd86e7d-3602-4e6d-baa4-40ae2ad9303c", resource_group="ManyModelsSA", workspace_name="ManyModelsSAv1")
ws.get_details()

experiment_name = 'automl-ojforecasting'
experiment = Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['SKU'] = ws.sku
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Run History Name'] = experiment_name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

,
SDK version,1.0.74
Subscription ID,bbd86e7d-3602-4e6d-baa4-40ae2ad9303c
Workspace,ManyModelsSAv1
SKU,Enterprise
Resource Group,ManyModelsSA
Location,centralus
Run History Name,automl-ojforecasting


## Compute

#### Creation of AmlCompute takes approximately 5 minutes. 
If the AmlCompute with that name is already in your workspace this code will skip the creation process.
As with other Azure services, there are limits on certain resources (e.g. AmlCompute) associated with the Azure Machine Learning service. Please read this article on the default limits and how to request more quota.

In [5]:

# Choose a name for your cluster.
amlcompute_cluster_name = "cpu-cluster"

found = False
# Check if this compute target already exists in the workspace.
cts = ws.compute_targets
if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
    found = True
    print('Found existing compute target.')
    compute_target = cts[amlcompute_cluster_name]
    
if not found:
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2", # for GPU, use "STANDARD_NC6"
                                                                #vm_priority = 'lowpriority', # optional
                                                                max_nodes = 6)

    # Create the cluster.
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)
    
print('Checking cluster status...')
# Can poll for a minimum number of nodes and for a specific timeout.
# If no min_node_count is provided, it will use the scale settings for the cluster.
compute_target.wait_for_completion(show_output = True, min_node_count = None, timeout_in_minutes = 20)
    
# For a more detailed view of current AmlCompute status, use get_status().

Found existing compute target.
Checking cluster status...
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


## Upload the raw data to the blob

Initially, we uploaded the raw data to blob storage from the local directory.

In [18]:
dataDir='raw_data/'
ojsalesraw_path=dataDir+'dominicks_OJ.csv'

In [19]:
datastore = ws.get_default_datastore() 
datastore.upload_files([ojsalesraw_path], target_path='ojsalesraw', overwrite=False, show_progress=True)

Uploading an estimated of 1 files
Target already exists. Skipping upload for ojsalesraw/dominicks_OJ.csv
Uploaded 0 files


$AZUREML_DATAREFERENCE_f5d895badb8848bda3be455b450efef0

## Dowloand the data from the blob

If you have the data in a blob container, you can download it to your local directory.

In [22]:
download_target_path = 'raw_data/'
datastore.download(target_path=download_target_path,
                   prefix='ojsalesraw/dominicks_OJ.csv',
                   show_progress=True)

WARNING - Path already exists. Skipping download for raw_data/ojsalesraw/dominicks_OJ.csv


0

## (Optional) Register dataset 

Register the dataset to the Workspace if needed. We skipped this step because we have ~12,000 datasets and we upload all our datasets into a blob container.

In [24]:

datastore_paths = [
                  (datastore, 'ojsalesraw')
                 ]
ojsalesraw_ds = Dataset.File.from_files(path=datastore_paths)

In [27]:
ojsalesraw_ds = ojsalesraw_ds.register(workspace=ws, 
                                       name='ojsalesraw', 
                                       description='ojsales raw data')

## Synthetic data generation

we are creating ~4,000 stores for the 2 years of the original dataset with columns Brand, Price, Advert, and Quantity.

In [3]:
time_column_name = 'WeekStarting'
data = pd.read_csv("./raw_data/dominicks_OJ.csv", parse_dates=[time_column_name])
data.head()

,WeekStarting,Store,Brand,Quantity,logQuantity,Advert,Price,Age60,COLLEGE,INCOME,Hincome150,Large HH,Minorities,WorkingWoman,SSTRDIST,SSTRVOL,CPDIST5,CPWVOL5
0,1990-06-14,2,dominicks,10560,9.26,1,1.59,0.23,0.25,10.55,0.46,0.10,0.11,0.30,2.11,1.14,1.93,0.38
1,1990-06-14,2,minute.maid,4480,8.41,0,3.17,0.23,0.25,10.55,0.46,0.10,0.11,0.30,2.11,1.14,1.93,0.38
2,1990-06-14,2,tropicana,8256,9.02,0,3.87,0.23,0.25,10.55,0.46,0.10,0.11,0.30,2.11,1.14,1.93,0.38
3,1990-06-14,5,dominicks,1792,7.49,1,1.59,0.12,0.32,10.92,0.54,0.10,0.05,0.41,3.80,0.68,1.60,0.74
4,1990-06-14,5,minute.maid,4224,8.35,0,2.99,0.12,0.32,10.92,0.54,0.10,0.05,0.41,3.80,0.68,1.60,0.74


In [4]:
data.describe()

,Store,Quantity,logQuantity,Advert,Price,Age60,COLLEGE,INCOME,Hincome150,Large HH,Minorities,WorkingWoman,SSTRDIST,SSTRVOL,CPDIST5,CPWVOL5
count,28947.00,28947.00,28947.00,28947.00,28947.00,28947.00,28947.00,28947.00,28947.00,28947.00,28947.00,28947.00,28947.00,28947.00,28947.00,28947.00
mean,80.88,17312.21,9.17,0.24,2.28,0.17,0.23,10.62,0.34,0.12,0.16,0.36,5.10,1.21,2.12,0.44
std,35.58,27477.66,1.02,0.43,0.65,0.06,0.11,0.28,0.24,0.03,0.19,0.05,3.47,0.53,0.73,0.22
min,2.00,64.00,4.16,0.00,0.52,0.06,0.05,9.87,0.00,0.01,0.02,0.24,0.13,0.40,0.77,0.09
25%,53.00,4864.00,8.49,0.00,1.79,0.12,0.15,10.46,0.12,0.10,0.04,0.31,2.77,0.73,1.63,0.27
50%,86.00,8384.00,9.03,0.00,2.17,0.17,0.23,10.64,0.35,0.11,0.07,0.36,4.65,1.12,1.96,0.38
75%,111.00,17408.00,9.76,0.00,2.73,0.21,0.28,10.80,0.53,0.14,0.19,0.40,6.65,1.54,2.53,0.56
max,137.00,716416.00,13.48,1.00,3.87,0.31,0.53,11.24,0.92,0.22,1.00,0.47,17.86,2.57,4.11,1.14


The original dataset's date range is from 1990-06-13 to 1992-10-02. We will generate the same date range and repeat that list for each store.

In [3]:
dates = pd.date_range(start ='1990-06-13', end ='1992-10-02', freq ='W-THU') 
dates_df = pd.DataFrame(data=dates)
dates_df_all = pd.DataFrame({'WeekStarting':np.repeat(dates_df.values,11973)})
dates_df_all.head()

,WeekStarting
0,1990-06-14
1,1990-06-14
2,1990-06-14
3,1990-06-14
4,1990-06-14


The original dataset's brands are dominicks, minute.maid, and tropicana. We will generate the same brands and repeat that list for each store.

In [4]:
brand_data={'Brand':['dominicks', 'minute.maid', 'tropicana']}
brands_list = pd.DataFrame(data=brand_data)
brands_df_all = pd.DataFrame()
brands_df_all = pd.concat([brands_list]*482911, ignore_index=True)
brands_df_all.head()

,Brand
0,dominicks
1,minute.maid
2,tropicana
3,dominicks
4,minute.maid


The store number is generated from the range of 138 to 4129 with increment of 1 to produce 3991 unique store numbers.

In [9]:
data.Store.unique()

array([  2,   5,   8,   9,  14,  18,  21,  28,  32,  33,  40,  44,  45,
        47,  48,  49,  50,  51,  52,  53,  54,  56,  59,  62,  64,  67,
        68,  70,  71,  72,  73,  75,  76,  77,  78,  80,  81,  83,  84,
        86,  88,  89,  90,  91,  92,  93,  94,  97, 101, 103, 104, 105,
       106, 107, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119,
       121, 122, 123, 124, 126, 128, 129, 132,  12,  74,  95,  98, 130,
       100, 102, 131, 137, 134])

In [5]:
store_list = np.arange(138,4129,1)
store_df = pd.DataFrame(data=store_list)
store_df_all = pd.DataFrame({'Store':np.repeat(store_df.values,3)})
store_df_final = pd.concat([store_df_all]*121, ignore_index=True)
store_df_final.head()

,Store
0,138
1,138
2,138
3,139
4,139


We combine the Date, Store, and Brand columns into a DataFrame

In [15]:
df_combined = pd.concat([dates_df_all, store_df_final, brands_df_all], axis=1, sort=False)

To maintain the distribution of the data and no outliers, the Quantity column is generated by selecting a random number between the 25th and 75th percentile from the original data. The price column is generated the same as Quantity using the 25th and 75th percentiles of the original data for the range.

In [18]:
df_combined['Quantity'] = np.random.randint(9000, 20000, df_combined.shape[0])
df_combined['Advert'] = np.random.randint(0,2)
df_combined['Price'] = np.random.randint(190, 270, df_combined.shape[0])/100
df_combined.head()

,WeekStarting,Store,Brand,Quantity,Advert,Price
0,1990-06-14,138,dominicks,13706,0,2.62
1,1990-06-14,138,minute.maid,12161,0,2.03
2,1990-06-14,138,tropicana,15080,0,2.29
3,1990-06-14,139,dominicks,16407,0,2.53
4,1990-06-14,139,minute.maid,13734,0,2.45


Last, we combine the original dataset to the generated dataset. The new dataset contains ~1.5 million records.

In [19]:
data_filtered = data[['WeekStarting', 'Store', 'Brand', 'Quantity','Advert', 'Price']]
frames = [data_filtered, df_combined]
df_final = pd.concat(frames)

We compared the summary statistics of the original data to the new dataset to confirm the means and standard deviations did not have a significant change.

In [20]:
data_filtered.describe()

,Store,Quantity,Advert,Price
count,28947.00,28947.00,28947.00,28947.00
mean,80.88,17312.21,0.24,2.28
std,35.58,27477.66,0.43,0.65
min,2.00,64.00,0.00,0.52
25%,53.00,4864.00,0.00,1.79
50%,86.00,8384.00,0.00,2.17
75%,111.00,17408.00,0.00,2.73
max,137.00,716416.00,1.00,3.87


In [21]:
df_final.describe()

,Store,Quantity,Advert,Price
count,1477680.00,1477680.00,1477680.00,1477680.00
mean,2092.80,14556.40,0.00,2.29
std,1175.69,4983.41,0.07,0.25
min,2.00,64.00,0.00,0.52
25%,1075.00,11663.00,0.00,2.09
50%,2093.00,14460.00,0.00,2.29
75%,3111.00,17255.00,0.00,2.50
max,4128.00,716416.00,1.00,3.87


Also adding a revenue column if you'd like to predict time-series revenue instead.

In [22]:
df_final['Revenue'] = df_final['Quantity']*df_final['Price'] 

## Save synthetic data and upload to blob

For the next steps, we will want to save the final output back to Blob Storage. We create a synthetic_data folder and upload the full dataset.

In [ ]:
synthetic_path = 'synthetic_data/'
syn_filename='ojsales.csv'

df_final.to_csv(path_or_buf=synthetic_path+syn_filename,index=False)

In [364]:
ojsalessynthetic_path='synthetic_data'
datastore.upload(src_dir=ojsalessynthetic_path,
                 target_path='ojsalessynthetic',
                 overwrite=True,
                 show_progress=True)

Uploading an estimated of 2 files
Uploading synthetic_data/.DS_Store
Uploading synthetic_data/ojsales.csv
Uploaded synthetic_data/.DS_Store, 1 files out of an estimated total of 2
Uploaded synthetic_data/ojsales.csv, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_028e1440013d4a1d8cec79a6c876eef6

## Save individual dataset and upload to blob

We also want the data broken into a csv for each store and brand. This is the granularity we will build our models at. To do this, we group the data by Store and Brand and save a csv. Then we upload those csv’s to Blob.

We have a total of 12,222 datasets uploaded. The time estimate is ~ 2 min to write them to local directory and ~2 min to uplaod to the blob.

In [ ]:
dfs = [x for _, x in df_final.groupby(['Store', 'Brand'])]

In [ ]:
refined_path = 'refined_data/'

for i in dfs:
    file_name = 'Store'+str(i['Store'].unique()).lstrip("['").rstrip("']")+'_'+str(i['Brand'].unique()).lstrip("['").rstrip("']")+'.csv'
    i.to_csv(path_or_buf=refined_path+file_name, index=False,)

In [ ]:
ojsalesrefined_path='refined_data'
datastore.upload(src_dir=ojsalesrefined_path,
                 target_path='ojsalesrefined',
                 overwrite=True,
                 show_progress=True)

## Next steps

1. Assign weights when generate random number between 0 and 1 to advert column.
2. Identify a less memeory consuming way to write 12,222 datasets to a local directory. Maybe a tmpdirectory and upload them to the blob.

In [ ]:
# advert=[1,0,0]
# advert_df = pd.DataFrame({'Advert':advert})
# advert_df_all = pd.DataFrame()
# advert_df_all = pd.concat([advert_df]*482911, ignore_index=True)